In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
transfer_filt = Filter.create_filter(address=None, event_types=[Pair.events.Transfer])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=transfer_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])

    if(len(arguments) >= 1):
        event['details']['transfer_value'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:61,383,742 tx:0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe724929bf6fcd64b3a2f8e7
Transfer at block:61,383,742 tx:0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe724929bf6fcd64b3a2f8e7
Transfer at block:61,383,742 tx:0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe724929bf6fcd64b3a2f8e7
Transfer at block:61,383,742 tx:0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe724929bf6fcd64b3a2f8e7
Transfer at block:61,383,742 tx:0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe724929bf6fcd64b3a2f8e7
Transfer at block:61,383,742 tx:0x65df1d1fd3534dadf652d72842361740c55d3f8653021148fb4ef7f151d44746
Transfer at block:61,383,742 tx:0x54949062140c6fdf9c174c45895c09bfa7feb05af0330c24edef410158333eed
Transfer at block:61,383,742 tx:0x2fdc851d8fafcf77aad3955bd0668374845bbfb28412132262431657e924969c
Transfer at block:61,383,742 tx:0x2fdc851d8fafcf77aad3955bd0668374845bbfb28412132262431657e924969c
Transfer at block:61,383,742 tx:0x2fdc851d8fafcf77aad3955bd0668374845bbfb28412132262431657e924969c
Transfer a

In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,transfer,sushi,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe72...,61383742,1725384947,{'web3_type': <class 'web3._utils.datatypes.Tr...
1,polygon,uniswapv2pair,transfer,sushi,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe72...,61383742,1725384947,{'web3_type': <class 'web3._utils.datatypes.Tr...
2,polygon,uniswapv2pair,transfer,sushi,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe72...,61383742,1725384947,{'web3_type': <class 'web3._utils.datatypes.Tr...
3,polygon,uniswapv2pair,transfer,sushi,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe72...,61383742,1725384947,{'web3_type': <class 'web3._utils.datatypes.Tr...
4,polygon,uniswapv2pair,transfer,sushi,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x5c826002e2b4becb23a827116a7a5b4a5c0c54acbe72...,61383742,1725384947,{'web3_type': <class 'web3._utils.datatypes.Tr...
...,...,...,...,...,...,...,...,...,...
5405,polygon,uniswapv2pair,transfer,sushi,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x7fc7131139d8dddd470f30c2bc22a37f885d2e87f956...,61383751,1725384967,{'web3_type': <class 'web3._utils.datatypes.Tr...
5406,polygon,uniswapv2pair,transfer,sushi,0x3c499c542cef5e3811e1192ce70d8cc03d5c3359,0x7fc7131139d8dddd470f30c2bc22a37f885d2e87f956...,61383751,1725384967,{'web3_type': <class 'web3._utils.datatypes.Tr...
5407,polygon,uniswapv2pair,transfer,sushi,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,0x7fc7131139d8dddd470f30c2bc22a37f885d2e87f956...,61383751,1725384967,{'web3_type': <class 'web3._utils.datatypes.Tr...
5408,polygon,uniswapv2pair,transfer,sushi,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,0x7fc7131139d8dddd470f30c2bc22a37f885d2e87f956...,61383751,1725384967,{'web3_type': <class 'web3._utils.datatypes.Tr...
